In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [103]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0, bias=False) #28x28x1 -> 26x26x8, RF:3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=0, bias=False) # 26x26x8 -> 24x24x8, RF:5
        self.bn2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=0, bias=False) # 24x24x8 -> 22x22x8, RF:7
        self.bn3 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2) # 11x11x8, RF:8
        self.conv4 = nn.Conv2d(8, 16, 3, padding=0, bias=False) # 11x11x8 -> 9x9x16, RF:12  
        self.bn4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 9x9x16 -> 7x7x16, RF:16 
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 7x7x16 -> 5x5x16, RF:20
        self.bn6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 5x5x16- > 3x3x16, RF:24
        self.bn7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 32, 3, padding=0, bias=False) # 3x3x16- > 1x1x16, RF:28
        #self.gap = nn.AvgPool2d(3) # 3x3x16 -> 1x1x16
        self.op = nn.Conv2d(32, 10, 1, 1, padding=0, bias=False)
        
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.bn3(F.relu(self.conv3(x)))
        
        x = self.pool1(x)
        
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.bn7(F.relu(self.conv7(x)))
        
        x = F.relu(self.conv8(x))
        #x = self.gap(x)
        x = self.op(x)
        
        x = x.squeeze(dim=(-2,-1))        
        return F.log_softmax(x, dim=-1)
        
        

In [104]:
model = Net().to(device)
xx = torch.randn(1,1,28,28).to(device)
yy = model(xx)
yy.squeeze(dim=(-1,-2)).shape

torch.Size([10])

In [105]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3            [-1, 8, 24, 24]             576
       BatchNorm2d-4            [-1, 8, 24, 24]              16
            Conv2d-5            [-1, 8, 22, 22]             576
       BatchNorm2d-6            [-1, 8, 22, 22]              16
         MaxPool2d-7            [-1, 8, 11, 11]               0
            Conv2d-8             [-1, 16, 9, 9]           1,152
       BatchNorm2d-9             [-1, 16, 9, 9]              32
           Conv2d-10             [-1, 16, 7, 7]           2,304
      BatchNorm2d-11             [-1, 16, 7, 7]              32
           Conv2d-12             [-1, 16, 5, 5]           2,304
      BatchNorm2d-13             [-1, 16, 5, 5]              32
           Conv2d-14             [-1, 1

In [106]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [107]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [108]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0723, Accuracy: 9774/10000 (97.740%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0366, Accuracy: 9879/10000 (98.790%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0194, Accuracy: 9942/10000 (99.420%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0178, Accuracy: 9942/10000 (99.420%)



loss=0.06351638585329056 batch_id=468: 100%|█████████████████████████████████████████| 469/469 [00:09<00:00, 49.01it/s]



Test set: Average loss: 0.0181, Accuracy: 9942/10000 (99.420%)

